<a href="https://colab.research.google.com/github/svecx/Report-Script/blob/staging/Final_Branch_Master_SAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
from datetime import datetime
import os

today = datetime.today().strftime('%d%m%y')

In [58]:
# def load_data():
#   try:
#     df_master = f"{today}_BRANCH_MASTER_SAP.XLSX"
#     df_lookup = "DWH_BRANCH 202508.xlsx"

#     if not os.path.exists(df_master):
#       raise FileNotFoundError(f"❌ File tidak ditemukan: {df_master}")
#     if not os.path.exists(df_master):
#       raise FileNotFoundError(f"❌ File tidak ditemukan: {df}")

#     df_master = pd.read_excel(df_master, keep_default_na=False)
#     df_lookup = pd.read_excel(df_lookup, sheet_name="Agustus 25", keep_default_na=False)
#     return df_master, df_lookup

#   except Exception as e:
#     print(f"⚠️ Gagal load data: {e}")


def load_data(df_title_manual=None):
    try:
        # Default lookup file (tetap di dalam fungsi)
        df_lookup_path = "DWH_BRANCH 202508.xlsx"

        # Tentukan tanggal
        today = datetime.today().strftime("%d%m%y")

        # Kalau ada input manual
        if df_title_manual:
            df_master_file = f"{df_title_manual}_BRANCH_MASTER_SAP.XLSX"
        else:
            # Kalau tidak ada, pakai otomatis (today)
            df_master_file = f"{today}_BRANCH_MASTER_SAP.XLSX"
            # contoh tambahan kalau mau pakai yesterday
            # src = f"Data {yesterday}_ZBRI_SRC_RO_SMNT.XLSX"

        # Validasi file
        if not os.path.exists(df_master_file):
            raise FileNotFoundError(f"❌ File tidak ditemukan: {df_master_file}")
        if not os.path.exists(df_lookup_path):
            raise FileNotFoundError(f"❌ File tidak ditemukan: {df_lookup_path}")

        # Load data
        df_master = pd.read_excel(df_master_file, keep_default_na=False)
        df_lookup = pd.read_excel(df_lookup_path, sheet_name="Agustus 25", keep_default_na=False)

        return df_master, df_lookup

    except Exception as e:
        print(f"⚠️ Gagal load data: {e}")
        return None, None

def merge_data(df_master, df_lookup):
  try:
    df_result = pd.merge(
        df_master,
        df_lookup[['SUBBR', 'MBNAME', 'RGNAME']],
        left_on="Branch ID - Originating Branch of Accoun",
        right_on="SUBBR",
        how="left"
    )
    df_result[['MBNAME', 'RGNAME']] = df_result[['MBNAME', 'RGNAME']].fillna("#NA")
    df_result = df_result[
        ['Branch ID - Originating Branch of Accoun','Single-Character Flag','Location of the Branch',
         'Revenue Owner','Business Segment','Changed by','Changed On','Time of change','MBNAME','RGNAME']
    ]
    return df_result
  except Exception as e:
    print(f"⚠️ Gagal merge data: {e}")
    return None


In [59]:
def generate_summary(df_master, df_result):
  try:
    filter_1 = df_master.groupby(
        ['Single-Character Flag','Revenue Owner','Business Segment'], dropna=False
    ).size().reset_index(name='Jumlah')

    filter_2 = filter_1[
        (filter_1['Single-Character Flag'] == 'U') & (
            (filter_1['Revenue Owner'].isna() | (filter_1['Revenue Owner'] == '')) |
            (filter_1['Business Segment'].isna() | (filter_1['Business Segment'] == ''))
        )
    ].shape[0]

    T_Filter3 = df_result[df_result['MBNAME'] == '#NA'].shape[0]
    T_Filter4 = df_result[df_result['RGNAME'] == '#NA'].shape[0]

    summary = pd.DataFrame({
        'Nama': ['Filter 2', 'T_Filter3', 'T_Filter4'],
        'Jumlah': [filter_2, T_Filter3, T_Filter4]
    })

    return summary, (filter_2, T_Filter3, T_Filter4)

  except Exception as e:
    print(f"⚠️ Gagal Filter Data: {e}")
    return None


def validate(filters):
  try:
    filter_2, T_Filter3, T_Filter4 = filters
    validations = [
        ("Filter_2 harus < 1", filter_2 < 1),
        ("T_Filter3 harus = 12", T_Filter3 == 12),
        ("T_Filter4 harus = 12", T_Filter4 == 12),
    ]
    invalids = [f"❌ {msg}" for msg, condition in validations if not condition]

    if invalids:
        print("📌 Masalah ditemukan:\n" + "\n".join(invalids))
    else:
        print("✅ Semua kondisi terpenuhi, data aman.")

  except Exception as e:
    print(f"⚠️ Gagal Validasi Data: {e}")
    return None


In [60]:
# def save_result(df_result):
#   try:
#     filename = f"Merge Branch Master Sap {today}.xlsx"
#     df_result.to_excel(filename, index=False)
#     print(f"📂 File berhasil disimpan: {filename}")

#   except Exception as e:
#     print(f"⚠️ Gagal Simpan Data: {e}")
#     return None

def save_result(df_result):
    try:
        today = datetime.today().strftime("%d%m%y")
        filename = f"Merge Branch Master Sap {today}.xlsx"

        # simpan dulu ke excel dengan openpyxl
        df_result.to_excel(filename, index=False, engine="openpyxl")

        # buka file excel untuk atur auto width
        wb = load_workbook(filename)
        ws = wb.active

        for col in ws.columns:
            max_length = 0
            col_letter = col[0].column_letter  # ambil nama kolom (A, B, C, dst.)

            for cell in col:
                try:
                    if cell.value:
                        length = len(str(cell.value))
                        if length > max_length:
                            max_length = length
                except Exception:
                    pass

            adjusted_width = max_length + 2  # kasih sedikit spasi
            ws.column_dimensions[col_letter].width = adjusted_width

        wb.save(filename)
        print(f"📂 File berhasil disimpan dengan auto width: {filename}")

    except Exception as e:
        print(f"⚠️ Gagal Simpan Data: {e}")
        return None

In [61]:
def main():
  # df_master, df_lookup = load_data("20250914")
  df_master, df_lookup = load_data()
  df_result = merge_data(df_master, df_lookup)
  summary, filters = generate_summary(df_master, df_result)
  print(summary)
  validate(filters)
  save_result(df_result)


main()

        Nama  Jumlah
0   Filter 2       0
1  T_Filter3      12
2  T_Filter4      12
✅ Semua kondisi terpenuhi, data aman.
⚠️ Gagal Simpan Data: name 'load_workbook' is not defined
